# Neuroharmonize datasets

This is a notebook to apply neuroHarmonize: a ComBat-GAM  non-linear allowing algorithm over our data to create neuroHarmonized datasets. It should be run in the `neurogamy` environment.

In [ ]:
import os
from neuroHarmonize import harmonizationLearn
import pandas as pd
import numpy as np

In [ ]:
Insight46_path = '../our_datasets/Insight46/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
SABRE_path = '../our_datasets/SABRE/'
EDIS_path = '../our_datasets/EDIS/'
HELIUS_path = '../our_datasets/HELIUS/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)
Insight46_file = os.path.join(Insight46_path, file_name)
HELIUS_file = os.path.join(HELIUS_path, file_name)

TOP = pd.read_csv(TOP_file)
MRI = pd.read_csv(MRI_file)
Insight46 = pd.read_csv(Insight46_file)
SABRE = pd.read_csv(SABRE_file)
EDIS = pd.read_csv(EDIS_file)

## the below must be changed to produce real HELIUS
HELIUS = pd.read_csv(EDIS_file)

In [ ]:
sex_mapping = {1:0,2:1}
Insight = Insight46.assign(Sex = Insight46.Sex.map(sex_mapping))
SABRE = SABRE.assign(Sex = SABRE.Sex.map(sex_mapping))
EDIS = EDIS.assign(Sex = EDIS.Sex.map(sex_mapping))
HELIUS = HELIUS.assign(Sex = HELIUS.Sex.map(sex_mapping))
Insight.head(3)

In [ ]:
HELIUS.tail(3)

In [ ]:
TOP['Site'] = 0
MRI['Site'] = 1
MRI.head(3)

In [ ]:
TOP.head(3)

# Here we will harmonize just TOP and StrokeMRI

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI.head(3) 

In [ ]:
TOPMRI.tail(3)

In [ ]:
TOPMRI.columns

In [ ]:
TOPMRI.columns= TOPMRI.columns.str.lower()

In [ ]:
common_features = ['gm_vol', 'wm_vol',
       'csf_vol', 'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf']

In [ ]:
TOPMRI_features = TOPMRI[common_features]
TOPMRI_covariates = TOPMRI[['age', 'sex','site']]
TOPMRI_covariates = TOPMRI_covariates.rename(columns={'site': 'SITE'})
TOPMRI_covariates.head(3)

In [ ]:
#TOPMRI_covariates.reset_index()

In [ ]:
TOPMRI_features_array = np.array(TOPMRI_features)
TOPMRI_features_array


In [ ]:
# run harmonization and PUT the adjusted data into my_ad_data vaiable
my_model, my_data_adj = harmonizationLearn(TOPMRI_features_array, TOPMRI_covariates)

In [ ]:
# turn adjusted data into dataframe with column names, then add covariates, then participant IDs
neuroharmonized_topmri = pd.DataFrame(
    my_data_adj, 
    columns = ['gm_vol', 'wm_vol',
       'csf_vol', 'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf']
)


neuroharmonized_topmri =pd.concat([neuroharmonized_topmri, TOPMRI_covariates.reset_index()], axis=1)
neuroharmonized_topmri = neuroharmonized_topmri.drop('index', axis=1)
neuroharmonized_topmri = pd.concat([neuroharmonized_topmri, TOPMRI.participant_id.reset_index()], axis=1)
neuroharmonized_topmri = neuroharmonized_topmri.drop('index', axis=1)
neuroharmonized_topmri.head(3)

In [ ]:
# create adjusted csvs
top_neuroharm_to_stroke = neuroharmonized_topmri[neuroharmonized_topmri.SITE == 0]
stroke_neuroharm_to_top = neuroharmonized_topmri[neuroharmonized_topmri.SITE == 1] 

In [ ]:
TOP.head(3)

In [ ]:
# reorganize to familair pattern
column_to_move1 = stroke_neuroharm_to_top.pop("participant_id")
column_to_move2 = stroke_neuroharm_to_top.pop("age")
column_to_move3 = stroke_neuroharm_to_top.pop("sex")
tcolumn_to_move1 = top_neuroharm_to_stroke.pop("participant_id")
tcolumn_to_move2 = top_neuroharm_to_stroke.pop("age")
tcolumn_to_move3 = top_neuroharm_to_stroke.pop("sex")
stroke_neuroharm_to_top.insert(0, "participant_id", column_to_move1)
stroke_neuroharm_to_top.insert(1, "age", column_to_move2)
stroke_neuroharm_to_top.insert(2, "sex", column_to_move3)
top_neuroharm_to_stroke.insert(0, "participant_id", tcolumn_to_move1)
top_neuroharm_to_stroke.insert(1, "age", tcolumn_to_move2)
top_neuroharm_to_stroke.insert(2, "sex", tcolumn_to_move3)

In [ ]:
top_neuroharm_to_stroke = top_neuroharm_to_stroke.drop('SITE', axis=1)
stroke_neuroharm_to_top = stroke_neuroharm_to_top.drop('SITE', axis=1)
stroke_neuroharm_to_top.columns

In [ ]:
top_neuroharm_to_stroke.to_csv('harm_results/neurocharm/top_neuroharm_to_stroke.csv')
stroke_neuroharm_to_top.to_csv('harm_results/neurocharm/stroke_neuroharm_to_top.csv')

In [ ]:
print(type(top_neuroharm_to_stroke))

In [ ]:
# (sabre_vs_topmri_only[['gm_vol', 'wm_vol', 'csf_vol',
#        'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
#        'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
#        'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',]] <0).sum()

# Now we join top and StrokeMRI to one dataset and harmonize to other datasets individually

In [ ]:
unified_TOPMRI = TOPMRI.copy(deep=True)
unified_TOPMRI = unified_TOPMRI.reset_index()
unified_TOPMRI['site'] = 0
unified_TOPMRI= unified_TOPMRI.drop('index', axis=1)
unified_TOPMRI.head(3)

In [ ]:
#SABRE = SABRE.drop('Unnamed: 0', axis=1)
#SABRE = SABRE.assign(sex = SABRE.sex.map(sex_mapping))
#SABRE['site'] = 2
SABRE.columns = SABRE.columns.str.lower()
SABRE.head(2) 

In [ ]:
SABRE['site'] = 2
SABRE.head(3)

In [ ]:
TOPMRIvsSABRE= pd.concat([unified_TOPMRI, SABRE])
TOPMRIvsSABRE = TOPMRIvsSABRE.reset_index()

In [ ]:
TOPMRIvsSABRE.columns

In [ ]:
TOPMRIvsSABRE_covariates = TOPMRIvsSABRE[['age', 'sex','site']]
TOPMRIvsSABRE_covariates = TOPMRIvsSABRE_covariates.rename(columns={'site': 'SITE'})
TOPMRIvsSABRE_covariates.head(3)

In [ ]:
 TOPMRIvsSABRE_features = TOPMRIvsSABRE[common_features]
TOPMRIvsSABRE_features_array = np.array(TOPMRIvsSABRE_features)

In [ ]:
my_model2, my_data_adj2 = harmonizationLearn(TOPMRIvsSABRE_features_array, TOPMRIvsSABRE_covariates)

In [ ]:
my_data_adj2

In [ ]:
# turn adjusted data into dataframe with column names, then add covariates, then participant IDs
neuroharmonized_topmrivsabre = pd.DataFrame(
    my_data_adj2, 
    columns = common_features
)
neuroharmonized_topmrivsabre =pd.concat([neuroharmonized_topmrivsabre, TOPMRIvsSABRE_covariates.reset_index()], axis=1)
neuroharmonized_topmrivsabre = neuroharmonized_topmrivsabre.drop('index', axis=1)
neuroharmonized_topmrivsabre = pd.concat([neuroharmonized_topmrivsabre, TOPMRIvsSABRE.participant_id.reset_index()], axis=1)
neuroharmonized_topmrivsabre = neuroharmonized_topmrivsabre.drop('index', axis=1)
neuroharmonized_topmrivsabre.head(3)

In [ ]:
# reorganize to familair pattern
column_to_move1 = neuroharmonized_topmrivsabre.pop("participant_id")
column_to_move2 = neuroharmonized_topmrivsabre.pop("age")
column_to_move3 = neuroharmonized_topmrivsabre.pop("sex")
neuroharmonized_topmrivsabre.insert(0, "participant_id", column_to_move1)
neuroharmonized_topmrivsabre.insert(1, "age", column_to_move2)
neuroharmonized_topmrivsabre.insert(2, "sex", column_to_move3)

In [ ]:
# create adjusted csvs
sabre_vs_topmri_only = neuroharmonized_topmrivsabre[neuroharmonized_topmrivsabre.SITE == 2]
topmri_vs_sabre_only = neuroharmonized_topmrivsabre[neuroharmonized_topmrivsabre.SITE == 0] 

In [ ]:
#topmri_vs_sabre_only

In [ ]:
sabre_vs_topmri_only = sabre_vs_topmri_only.drop('SITE', axis=1)
topmri_vs_sabre_only  = topmri_vs_sabre_only .drop('SITE', axis=1)
#topmri_vs_sabre_only.columns

In [ ]:
#sabre_vs_topmri_only

In [ ]:
sabre_vs_topmri_only.to_csv('harm_results/neurocharm/sabre_vs_topmri_only.csv') 
topmri_vs_sabre_only.to_csv('harm_results/neurocharm/topmri_vs_sabre_only.csv') 

In [ ]:
#

In [ ]:
Insight46['Site'] = 3
Insight46.head(2) 

In [ ]:
Insight46.columns = Insight46.columns.str.lower()

In [ ]:
TOPMRIvsInsight46= pd.concat([unified_TOPMRI, Insight46])
TOPMRIvsInsight46 = TOPMRIvsInsight46.reset_index()

In [ ]:
TOPMRIvsInsight_covariates = TOPMRIvsInsight46[['age', 'sex','site']]
TOPMRIvsInsight_covariates = TOPMRIvsInsight_covariates .rename(columns={'site': 'SITE'})
TOPMRIvsInsight_covariates.head(3)

In [ ]:
 TOPMRIvsInsight_features = TOPMRIvsInsight46[common_features]
TOPMRIvsInsight_features_array = np.array(TOPMRIvsInsight_features)

In [ ]:
my_model3, my_data_adj3 = harmonizationLearn(TOPMRIvsInsight_features_array, TOPMRIvsInsight_covariates)

In [ ]:
my_data_adj3

In [ ]:
# turn adjusted data into dataframe with column names, then add covariates, then participant IDs
neuroharmonized_topmrivinsight = pd.DataFrame(
    my_data_adj3, 
    columns = common_features
)
neuroharmonized_topmrivinsight = pd.concat([neuroharmonized_topmrivinsight, TOPMRIvsInsight_covariates.reset_index()], axis=1)
neuroharmonized_topmrivinsight = neuroharmonized_topmrivinsight.drop('index', axis=1)
neuroharmonized_topmrivinsight = pd.concat([neuroharmonized_topmrivinsight, TOPMRIvsInsight46.participant_id.reset_index()], axis=1)
neuroharmonized_topmrivinsight = neuroharmonized_topmrivinsight.drop('index', axis=1)
neuroharmonized_topmrivinsight.head(3)

In [ ]:
#neuroharmonized_topmrivinsight

In [ ]:
# reorganize to familair pattern
column_to_move1 = neuroharmonized_topmrivinsight.pop("participant_id")
column_to_move2 = neuroharmonized_topmrivinsight.pop("age")
column_to_move3 = neuroharmonized_topmrivinsight.pop("sex")
neuroharmonized_topmrivinsight.insert(0, "participant_id", column_to_move1)
neuroharmonized_topmrivinsight.insert(1, "age", column_to_move2)
neuroharmonized_topmrivinsight.insert(2, "sex", column_to_move3)

In [ ]:
# create adjusted csvs
insight_vs_topmri_only = neuroharmonized_topmrivinsight[neuroharmonized_topmrivinsight.SITE == 3]
topmri_vs_insight_only = neuroharmonized_topmrivinsight[neuroharmonized_topmrivinsight.SITE == 0] 

In [ ]:
insight_vs_topmri_only  = insight_vs_topmri_only.drop('SITE', axis=1)
topmri_vs_insight_only  = topmri_vs_insight_only.drop('SITE', axis=1)


In [ ]:

#insight_vs_topmri_only

In [ ]:
insight_vs_topmri_only.to_csv('harm_results/neurocharm/insight_vs_topmri_only.csv') 
topmri_vs_insight_only.to_csv('harm_results/neurocharm/topmri_vs_insight_only.csv') 

In [ ]:
EDIS['Site'] = 4
EDIS.head(2) 

In [ ]:
EDIS.columns = EDIS.columns.str.lower()

In [ ]:
TOPMRIvsEDIS = pd.concat([unified_TOPMRI, EDIS])
TOPMRIvsEDIS = TOPMRIvsEDIS.reset_index()

In [ ]:
TOPMRIvsEDIS_covariates = TOPMRIvsEDIS[['age', 'sex','site']]
TOPMRIvsEDIS_covariates = TOPMRIvsEDIS_covariates .rename(columns={'site': 'SITE'})
TOPMRIvsEDIS_covariates.head(3)

In [ ]:
TOPMRIvsEDIS_features = TOPMRIvsEDIS[common_features]
TOPMRIvsEDIS_features_array = np.array(TOPMRIvsEDIS_features)

In [ ]:
my_model4, my_data_adj4 = harmonizationLearn(TOPMRIvsEDIS_features_array, TOPMRIvsEDIS_covariates)

In [ ]:
my_data_adj4

In [ ]:
# turn adjusted data into dataframe with column names, then add covariates, then participant IDs
neuroharmonized_topmrivedis = pd.DataFrame(
    my_data_adj4, 
    columns = common_features
)
neuroharmonized_topmrivedis = pd.concat([neuroharmonized_topmrivedis, TOPMRIvsEDIS_covariates.reset_index()], axis=1)
neuroharmonized_topmrivedis = neuroharmonized_topmrivedis.drop('index', axis=1)
neuroharmonized_topmrivedis = pd.concat([neuroharmonized_topmrivedis, TOPMRIvsEDIS.participant_id.reset_index()], axis=1)
neuroharmonized_topmrivedis = neuroharmonized_topmrivedis.drop('index', axis=1)
neuroharmonized_topmrivedis.head(3)

In [ ]:
# reorganize to familair pattern
column_to_move1 = neuroharmonized_topmrivedis.pop("participant_id")
column_to_move2 = neuroharmonized_topmrivedis.pop("age")
column_to_move3 = neuroharmonized_topmrivedis.pop("sex")
neuroharmonized_topmrivedis.insert(0, "participant_id", column_to_move1)
neuroharmonized_topmrivedis.insert(1, "age", column_to_move2)
neuroharmonized_topmrivedis.insert(2, "sex", column_to_move3)

In [ ]:
# create adjusted csvs
edis_vs_topmri_only = neuroharmonized_topmrivedis[neuroharmonized_topmrivedis.SITE == 4]
topmri_vs_edis_only = neuroharmonized_topmrivedis[neuroharmonized_topmrivedis.SITE == 0] 

In [ ]:
edis_vs_topmri_only  = edis_vs_topmri_only.drop('SITE', axis=1)
topmri_vs_edis_only  = topmri_vs_edis_only.drop('SITE', axis=1)


In [ ]:
#edis_vs_topmri_only

In [ ]:
edis_vs_topmri_only.to_csv('harm_results/neurocharm/edis_vs_topmri_only.csv') 
topmri_vs_edis_only.to_csv('harm_results/neurocharm/topmri_vs_edis_only.csv') 

In [ ]:
HELIUS['Site'] = 5
HELIUS.head(2) 

In [ ]:
HELIUS.columns = HELIUS.columns.str.lower()

In [ ]:
TOPMRIvsHELIUS = pd.concat([unified_TOPMRI, HELIUS])
TOPMRIvsHELIUS = TOPMRIvsHELIUS.reset_index()

In [ ]:
TOPMRIvsHELIUS_covariates = TOPMRIvsHELIUS[['age', 'sex','site']]
TOPMRIvsHELIUS_covariates = TOPMRIvsHELIUS_covariates .rename(columns={'site': 'SITE'})
TOPMRIvsHELIUS_covariates.head(3)

In [ ]:
TOPMRIvsHELIUS_features = TOPMRIvsHELIUS[common_features]
TOPMRIvsHELIUS_features_array = np.array(TOPMRIvsHELIUS_features)

In [ ]:
my_model5, my_data_adj5 = harmonizationLearn(TOPMRIvsHELIUS_features_array, TOPMRIvsHELIUS_covariates)

In [ ]:
my_data_adj5

In [ ]:
# turn adjusted data into dataframe with column names, then add covariates, then participant IDs
neuroharmonized_topmrivhelius = pd.DataFrame(
    my_data_adj5, 
    columns = common_features
)
neuroharmonized_topmrivhelius = pd.concat([neuroharmonized_topmrivhelius, TOPMRIvsHELIUS_covariates.reset_index()], axis=1)
neuroharmonized_topmrivhelius = neuroharmonized_topmrivhelius.drop('index', axis=1)
neuroharmonized_topmrivhelius = pd.concat([neuroharmonized_topmrivhelius, TOPMRIvsHELIUS.participant_id.reset_index()], axis=1)
neuroharmonized_topmrivhelius = neuroharmonized_topmrivhelius.drop('index', axis=1)
neuroharmonized_topmrivhelius.head(3)

In [ ]:
# reorganize to familair pattern
column_to_move1 = neuroharmonized_topmrivhelius.pop("participant_id")
column_to_move2 = neuroharmonized_topmrivhelius.pop("age")
column_to_move3 = neuroharmonized_topmrivhelius.pop("sex")
neuroharmonized_topmrivhelius.insert(0, "participant_id", column_to_move1)
neuroharmonized_topmrivhelius.insert(1, "age", column_to_move2)
neuroharmonized_topmrivhelius.insert(2, "sex", column_to_move3)

In [ ]:
neuroharmonized_topmrivhelius

In [ ]:
# create adjusted csvs
helius_vs_topmri_only = neuroharmonized_topmrivhelius[neuroharmonized_topmrivhelius.SITE == 5]
topmri_vs_helius_only = neuroharmonized_topmrivhelius[neuroharmonized_topmrivhelius.SITE == 0] 

In [ ]:
helius_vs_topmri_only  = helius_vs_topmri_only.drop('SITE', axis=1)
topmri_vs_helius_only  = topmri_vs_helius_only.drop('SITE', axis=1)


In [ ]:
helius_vs_topmri_only

In [ ]:
helius_vs_topmri_only.to_csv('harm_results/neurocharm/helius_vs_topmri_only.csv') 
topmri_vs_helius_only.to_csv('harm_results/neurocharm/topmri_vs_helius_only.csv') 

# Now we will do five way harmonization- TOPMRI vs. SAbre vs. Insigh46 vs. EDIS vs. others

In [ ]:
#this is all pending from here forward, we await the permissions on the other dataset

In [ ]:
TOPMRIvsSABRvsInisghvsEDISvsHELIUS = pd.concat([unified_TOPMRI, SABRE, Insight46, EDIS, HELIUS])
TOPMRIvsSABRvsInisghvsEDISvsHELIUS = TOPMRIvsSABRvsInisghvsEDISvsHELIUS.reset_index()

In [ ]:
TOPMRIvsSABRvsInisghvsEDISvsHELIUS_covariates = TOPMRIvsSABRvsInisghvsEDISvsHELIUS[['age', 'sex','site']]
TOPMRIvsSABRvsInisghvsEDISvsHELIUS_covariates = TOPMRIvsSABRvsInisghvsEDISvsHELIUS_covariates.rename(columns={'site': 'SITE'})
TOPMRIvsSABRvsInisghvsEDISvsHELIUS_covariates.head(3)

In [ ]:
TOPMRIvsSABRvsInisghvsEDISvsHELIUS.columns

In [ ]:
TOPMRIvsSABRvsInisghvsEDISvsHELIUS_features = TOPMRIvsSABRvsInisghvsEDISvsHELIUS[[ 
     'gm_vol',
       'wm_vol', 'csf_vol', 'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol',
       'wmh_count', 'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov',
       'totalgm_b_cov', 'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf',
       'totalgm_b_cbf'
]]
TOPMRIvsSABRvsInisghvsEDISvsHELIUS_features_array = np.array(TOPMRIvsSABRvsInisghvsEDISvsHELIUS_features)

In [ ]:
my_modelA, my_data_adjA = harmonizationLearn(TOPMRIvsSABRvsInisghvsEDISvsHELIUS_features_array, TOPMRIvsSABRvsInisghvsEDISvsHELIUS_covariates)

In [ ]:
my_data_adjA

In [ ]:
# turn adjusted data into dataframe with column names, then add covariates, then participant IDs
neuroharmonized_A= pd.DataFrame(
    my_data_adjA, 
    columns = ['gm_vol',
       'wm_vol', 'csf_vol', 'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol',
       'wmh_count', 'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov',
       'totalgm_b_cov', 'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf',
       'totalgm_b_cbf']
)
neuroharmonized_A =pd.concat([neuroharmonized_A, TOPMRIvsSABRvsInisghvsEDISvsHELIUS_covariates .reset_index()], axis=1)
neuroharmonized_A = neuroharmonized_A.drop('index', axis=1)
neuroharmonized_A = pd.concat([neuroharmonized_A, TOPMRIvsSABRvsInisghvsEDISvsHELIUS.participant_id.reset_index()], axis=1)
neuroharmonized_A = neuroharmonized_A.drop('index', axis=1)
neuroharmonized_A.head(3)

In [ ]:
# reorganize to familair pattern
column_to_move1 = neuroharmonized_A.pop("participant_id")
column_to_move2 = neuroharmonized_A.pop("age")
column_to_move3 = neuroharmonized_A.pop("sex")
neuroharmonized_A.insert(0, "participant_id", column_to_move1)
neuroharmonized_A.insert(1, "age", column_to_move2)
neuroharmonized_A.insert(2, "sex", column_to_move3)

In [ ]:
# create adjusted csvs
insight_vs_topmri_5way= neuroharmonized_A[neuroharmonized_A.SITE == 3]
sabre_vs_topmri_5way = neuroharmonized_A[neuroharmonized_A.SITE == 2]
topmri_vs_sabre_5way = neuroharmonized_A[neuroharmonized_A.SITE == 0] 
edis_vs_topmri_5way= neuroharmonized_A[neuroharmonized_A.SITE == 4]
helius_vs_topmri_5way = neuroharmonized_A[neuroharmonized_A.SITE == 5]

In [ ]:
insight_vs_topmri_5way = insight_vs_topmri_5way.drop('SITE', axis=1)
sabre_vs_topmri_5way = sabre_vs_topmri_5way.drop('SITE', axis=1) 
topmri_vs_sabre_5way = topmri_vs_sabre_5way.drop('SITE', axis=1)  

edis_vs_topmri_5way = edis_vs_topmri_5way.drop('SITE', axis=1) 
helius_vs_topmri_5way = helius_vs_topmri_5way.drop('SITE', axis=1) 

In [ ]:
helius_vs_topmri_5way 

In [ ]:
insight_vs_topmri_5way.to_csv('harm_results/neurocharm/insight_vs_topmri_5way.csv')
helius_vs_topmri_5way.to_csv('harm_results/neurocharm/helius_vs_topmri_5way.csv')
sabre_vs_topmri_5way.to_csv('harm_results/neurocharm/sabre_vs_topmri_5way.csv')
topmri_vs_sabre_5way.to_csv('harm_results/neurocharm/topmri_vs_sabre_5way.csv')
edis_vs_topmri_5way.to_csv('harm_results/neurocharm/edis_vs_topmri_5way.csv')